In [1]:
pip install imagecodecs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 40.8 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm


# Input directories
INPUT_IMAGES_DIR = "/kaggle/input/dataset2/standalone_dataset_1848/images"      
INPUT_MASKS_DIR = "/kaggle/input/dataset2/standalone_dataset_1848/masks"        

# Output directories
OUTPUT_IMAGES_DIR = "./images"        
OUTPUT_MASKS_DIR = "./masks"          

# Tiling parameters
TILE_SIZE = 256                  # Size of each tile (256x256)
ORIGINAL_SIZE = 2048             # Size of original images (2048x2048)


def create_tiles(image_path, mask_path, output_image_dir, output_mask_dir, 
                tile_size=256, original_size=2048):
    """
    Split a large image and its corresponding mask into smaller tiles without overlap.
    """
    
    # Load image and mask
    try:
        # Load image (convert to RGB for consistency)
        image = Image.open(image_path).convert('RGB')
        
        # Load mask (always grayscale)
        mask = Image.open(mask_path).convert('L')
            
    except Exception as e:
        print(f"Error loading {image_path} or {mask_path}: {e}")
        return []
    
    
    # Get base filename without extension
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    
    # Calculate number of tiles in each dimension (simple division without overlap)
    num_tiles_per_dimension = original_size // tile_size
    
    saved_tiles = []
    
    for i in range(num_tiles_per_dimension):
        for j in range(num_tiles_per_dimension):
            # Calculate tile coordinates
            x1 = i * tile_size
            y1 = j * tile_size
            x2 = x1 + tile_size
            y2 = y1 + tile_size
            
            # Extract tiles
            image_tile = image.crop((x1, y1, x2, y2))
            mask_tile = mask.crop((x1, y1, x2, y2))
            
            # Create tile filename
            tile_name = f"{base_name}_tile_{i:02d}_{j:02d}"
            
            # Save image tile as PNG
            image_tile_path = os.path.join(output_image_dir, f"{tile_name}.png")
            image_tile.save(image_tile_path)
            
            # Save mask tile as PNG
            mask_tile_path = os.path.join(output_mask_dir, f"{tile_name}.png")
            mask_tile.save(mask_tile_path)
            
            saved_tiles.append(tile_name)
    
    return saved_tiles

def process_dataset(input_image_dir, input_mask_dir, output_image_dir, output_mask_dir,
                   tile_size=256, original_size=2048):
    """
    Process an entire dataset of images and masks.
    """
    
    # Create output directories if they don't exist
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_mask_dir, exist_ok=True)
    
    # Get list of image files 
    image_extensions = ('.jpg', '.png', '.tiff', '.tif')
    image_files = [f for f in os.listdir(input_image_dir) 
                   if f.lower().endswith(image_extensions)]
    
    # Get list of mask files
    mask_files = [f for f in os.listdir(input_mask_dir) 
                  if f.lower().endswith(image_extensions)]
    
    # Create mapping of base names to full filenames
    image_map = {os.path.splitext(f)[0]: f for f in image_files}
    mask_map = {os.path.splitext(f)[0]: f for f in mask_files}
    
    # Find matching pairs
    matching_pairs = []
    for base_name in image_map:
        if base_name in mask_map:
            matching_pairs.append((
                os.path.join(input_image_dir, image_map[base_name]),
                os.path.join(input_mask_dir, mask_map[base_name])
            ))
        else:
            print(f"Warning: No matching mask found for image {image_map[base_name]}")
    
    print(f"All tiles will be saved as PNG files")
    
    
    total_tiles_saved = 0
    
    # Process each pair
    for image_path, mask_path in tqdm(matching_pairs, desc="Processing images"):
        saved_tiles = create_tiles(
            image_path, mask_path, 
            output_image_dir, output_mask_dir,
            tile_size, original_size
        )
        total_tiles_saved += len(saved_tiles)
        
        if len(saved_tiles) == 0:
            print(f"Warning: No tiles saved for {os.path.basename(image_path)}")
    
    print(f"\nProcessing completed!")
    print(f"Total tiles saved: {total_tiles_saved}")
    print(f"Image tiles saved in: {output_image_dir}")
    print(f"Mask tiles saved in: {output_mask_dir}")

def main():
    print("Image Tiling Script")
    print("-------------------")
    print(f"Input images directory: {INPUT_IMAGES_DIR}")
    print(f"Input masks directory: {INPUT_MASKS_DIR}")
    print(f"Output images directory: {OUTPUT_IMAGES_DIR}")
    print(f"Output masks directory: {OUTPUT_MASKS_DIR}")
    print(f"Tile size: {TILE_SIZE}x{TILE_SIZE}")
    print(f"Original size: {ORIGINAL_SIZE}x{ORIGINAL_SIZE}")
    
    # Process the dataset
    process_dataset(
        INPUT_IMAGES_DIR, INPUT_MASKS_DIR,
        OUTPUT_IMAGES_DIR, OUTPUT_MASKS_DIR,
        TILE_SIZE, ORIGINAL_SIZE
    )

if __name__ == "__main__":
    main()

Image Tiling Script
-------------------
Input images directory: /kaggle/input/dataset2/standalone_dataset_1848/images
Input masks directory: /kaggle/input/dataset2/standalone_dataset_1848/masks
Output images directory: ./images
Output masks directory: ./masks
Tile size: 256x256
Original size: 2048x2048
All tiles will be saved as PNG files


Processing images:   0%|          | 2/1848 [00:04<1:08:17,  2.22s/it]